# Übung 1 vom 11.10.2022

Sie besitzen einen Rucksack mit einer Kapazität von n Kilogramm. <br>
Sie wollen nun eine Reihe von Produkten in den Rucksack packen, so dass der Wert der gepackten Produkte maximal ist.


### Erläuterung Dataclasses

Dataclasses sind Klassen, die als Daten-Container verwendet werden können. <br>
Die Funktion `__post_init__()` wird nach der Initialisierung der Klasse aufgerufen und setzt zusätzliche berechnete Felder. <br>
Beispiel: Die Variable 'value' wird nicht initialisiert(`init=False`) und nicht bei einem print angezeigt(`repr=False`) <br>
```py
value: float = field(init=False, repr=False)
```

### Product Dataclass

In [274]:
from dataclasses import dataclass, field

@dataclass
class Product:
    name: str
    weight: float
    value: float
    multiplier: float = 1
    ratio: float = field(init=False)
    
    def rel_value(self) -> float:
        return round(self.value * self.multiplier, 2)
    
    def rel_weight(self) -> float:
        return round(self.weight * self.multiplier, 2)
    
    def __post_init__(self) -> None:
        self.ratio = round(self.value / self.weight, 2)

### Bag Dataclass

In [275]:
@dataclass
class Bag:
    name: str
    capacity: float
    products: list[Product]

    def total_weight(self) -> float:
        return round(sum([(p.weight * p.multiplier) for p in self.products]), 2)

    def total_value(self) -> float:
        return round(sum([(p.value * p.multiplier) for p in self.products]), 2)

    def __str__(self) -> str:
        products_str = "\n  - ".join([f"{p.multiplier}x\t{p.name}({p.rel_value()}€, {p.rel_weight()}kg, {p.ratio}€/kg)" for p in self.products])
        return (f"Bag: {self.name}\n"
        f"- Capacity: {self.total_weight()}kg / {self.capacity}kg\n"
        f"- Value: {self.total_value()}€\n"
        f"- Products: {len(self.products)}\n  - {products_str}")

Set the capacity of the bags and initialize the given products.

In [276]:
# default bag capacity in kg
capacity = 41

# generate products
products = [Product("Product1", 12.34, 123.99),
            Product("Product2", 23.45, 600.54),
            Product("Product3", 12.78, 90.67),
            Product("Product4", 9.34, 34.32)]

# List all products sorted by value/weight ratio
for p in sorted(products, key=lambda p: p.ratio, reverse=True):
    print(p)

Product(name='Product2', weight=23.45, value=600.54, multiplier=1, ratio=25.61)
Product(name='Product1', weight=12.34, value=123.99, multiplier=1, ratio=10.05)
Product(name='Product3', weight=12.78, value=90.67, multiplier=1, ratio=7.09)
Product(name='Product4', weight=9.34, value=34.32, multiplier=1, ratio=3.67)


### Exercise 1 - The fractal algorithm  

In [277]:
bag = Bag("Fractal Bag", capacity, [])
temp_products = products.copy()

# add products to bag until capacity is reached
while bag.total_weight() < bag.capacity:
    # exit if no products are left
    if len(temp_products) == 0:
        break
    # get product with highest value/weight ratio
    product = max(temp_products, key=lambda x: x.ratio)
    # check if capacity would be reached
    if bag.total_weight() + product.weight >= bag.capacity:
        # add last product with adjusted multiplier
        product.multiplier = round(
            (bag.capacity - bag.total_weight()) / product.weight, 4)
        bag.products.append(product)
        break
    # add product to bag
    bag.products.append(product)
    # remove product from list
    temp_products.remove(product)

print(bag)

Bag: Fractal Bag
- Capacity: 41.0kg / 41kg
- Value: 761.5€
- Products: 3
  - 1x Product2(600.54€, 23.45kg, 25.61€/kg)
  - 1x Product1(123.99€, 12.34kg, 10.05€/kg)
  - 0.4077x Product3(36.97€, 5.21kg, 7.09€/kg)


### Exercise 2 - The discrete algorithm (optimal?)

In [278]:
bag = Bag("Discrete Bag", capacity, [])
temp_products = products.copy()

# add products to bag until capacity is reached
while bag.total_weight() < bag.capacity:
    # exit if no products are left
    if len(temp_products) == 0:
        break
    # get product with highest value/weight ratio
    product = max(temp_products, key=lambda x: x.ratio)
    # check if capacity would be reached
    if bag.total_weight() + product.weight >= bag.capacity:
        break
    # add product to bag
    bag.products.append(product)
    # remove product from list
    temp_products.remove(product)
    
print(bag)

Bag: Discrete Bag
- Capacity: 35.79kg / 41kg
- Value: 724.53€
- Products: 2
  - 1x Product2(600.54€, 23.45kg, 25.61€/kg)
  - 1x Product1(123.99€, 12.34kg, 10.05€/kg)
